## Extract Data

In [1]:
# importing required modules
from zipfile import ZipFile

# ***** RUN THIS ONCE *****
# specifying the zip file name
# file_name = "spotify_million_playlist_dataset.zip"

# opening the zip file in READ mode
# with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    # zip.printdir()
#     i = 0
#     print('Extracting all the files now...')
#     for name in zip.namelist():
#         if i > 20:
#             break
        
        # get the first 20 data files
 #        if name.startswith("data/m"):
#             zip.extract(name)
#             print(name)
#             i += 1
       
#     print('Done!')
    # extracting all the files
    # print('Extracting all the files now...')
    # zip.extractall()
    # print('Done!')

In [2]:
import os
import json
def loop_slices(path, num_slices=20):
    cnt = 0
    mpd_playlists = []
    filenames = os.listdir(path)
    for fname in sorted(filenames):
        print(fname)
        if fname.startswith("mpd.slice.") and fname.endswith(".json"):
            cnt += 1
            fullpath = os.sep.join((path, fname))
            f = open(fullpath)
            js = f.read()
            f.close()
            current_slice = json.loads(js)
            # Create a list of all playlists
            for playlist in current_slice['playlists']:
                mpd_playlists.append(playlist)
            if cnt == num_slices:
                break
    return mpd_playlists
# Path where the json files are extracted
path = 'data/'
playlists = loop_slices(path, num_slices=20)

mpd.slice.115000-115999.json
mpd.slice.193000-193999.json
mpd.slice.225000-225999.json
mpd.slice.290000-290999.json
mpd.slice.324000-324999.json
mpd.slice.38000-38999.json
mpd.slice.391000-391999.json
mpd.slice.422000-422999.json
mpd.slice.448000-448999.json
mpd.slice.497000-497999.json
mpd.slice.523000-523999.json
mpd.slice.549000-549999.json
mpd.slice.596000-596999.json
mpd.slice.613000-613999.json
mpd.slice.679000-679999.json
mpd.slice.695000-695999.json
mpd.slice.7000-7999.json
mpd.slice.712000-712999.json
mpd.slice.778000-778999.json
mpd.slice.875000-875999.json


## Get Spotify API Data

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
# from tqdm import tqdm
# Spotify credentials
os.environ["SPOTIPY_CLIENT_ID"] = "bdf64242b8364ab5b264d3c14e8e9af6"
os.environ["SPOTIPY_CLIENT_SECRET"] = "3ed931eb80d8412292a50a10ed96e611"
os.environ['SPOTIPY_REDIRECT_URI'] = "http://localhost:5001"
sp = spotipy.Spotify(client_credentials_manager =      
                     SpotifyClientCredentials())
cols_to_keep = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
dfs = []
for playlist in playlists[0:5]:
    audio_feats = []
    for track in playlist['tracks']:
        track_uri = track['track_uri'].split(":")[2]
        feature = sp.audio_features(track_uri)
        if feature:
            audio_feats.append(feature[0])
    feats = pd.DataFrame(audio_feats)[cols_to_keep]
    feats['name'] = playlist['name']
    feats['pid'] = playlist['pid']
    # avg_feats = pd.DataFrame(audio_feats)[cols_to_keep].mean()
    # avg_feats['name'] = playlist['name']
    # avg_feats['pid'] = playlist['pid']
    dfs.append(feats.T)